In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

import os, shutil, subprocess, json, sys
from importlib import reload
from joblib import Parallel, delayed

import msprime, tskit

from pathlib import Path
sys.path.append(str(Path.cwd().parent))
import sims

# Visualization defaults
sns.set_theme(style="whitegrid", palette="colorblind")
plt.rcParams['figure.figsize'] = (10, 6)

In [3]:
# parameters for simulation
params = {
    'chrom_length': 5e7,      # DNA sequence length in base pairs
    'recomb_rate': 1e-8,      # Recombination rate (per bp per generation)
    'mut_rate': 1.29e-8,      # Mutation rate (per bp per generation)
    'gen_time': 29.0,         # Years per generation
    'n_eu': 1,          # European samples
    'n_af': 200,         # African samples
    'n_nd': 10,      # Neanderthal sample number
    'ploidy' : 2
}

# effective pop sizes in demography
Ne = {
    'anc': 18500,      # Common AMH-Neanderthal ancestor
    'nd': 3400,     # Neanderthal population
    'amh': 23000,           # Anatomically modern humans (pre-OoA)
    'ooa': 1861,            # Out-of-Africa founders
    'af': 27600,       # African population
    'eu': 13377,      # European population
    'eu_bottleneck': 1080,  # European bottleneck size
    'eu_growth': 1450       # Post-bottleneck size
}

# split and migration times
t = {
    't_nd_migration': 55000 / params['gen_time'],    # Neanderthal migration into OOA
    't_amh': 550000 / params['gen_time'],         # AMH-Neanderthal split  
    't_ooa': 65700 / params['gen_time'],             # Out-of-Africa migration
    't_nd_samples': 38000 / params['gen_time'],      # Neanderthal sampling time
    't_eu_growth': 31900 / params['gen_time'],          # European growth start
}

OUTPUT_DIR = "sims"

# Проверяем, существует ли папка
if os.path.exists(OUTPUT_DIR):
    print(f"DIR '{OUTPUT_DIR}' exists already. Clean up old files...")
    shutil.rmtree(OUTPUT_DIR) # Удаляет папку рекурсивно вместе со всеми файлами
os.makedirs(OUTPUT_DIR, exist_ok=True)

N_CHR, N_JOBS = 10, 4  # number of chr, -1 use all cores, N use N cores
random_seeds = random.sample(range(1, 10**9), k=N_CHR)

DIR 'sims' exists already. Clean up old files...


In [4]:
results = Parallel(n_jobs=N_JOBS)(
    delayed(sims.process_one_chromosome)(seed, params, Ne, t, 0.03, OUTPUT_DIR) 
    for seed in random_seeds
)
print(f"All simulations are finished...All trees are in {OUTPUT_DIR}/")

final_df = pd.concat(results, ignore_index=True)
csv_path = os.path.join(OUTPUT_DIR, "all.tracts.summary.csv")
final_df.to_csv(csv_path, index=False, sep='\t')

print(f"Nd in EU tract table in {csv_path}")

All simulations are finished...All trees are in sims/
Nd in EU tract table in sims/all.tracts.summary.csv


In [5]:
print(f"Running DAIseg in parallel for {len(random_seeds)} chromosomes...")
# Run in parallel
results = Parallel(n_jobs=N_JOBS)(
    delayed(sims.run_daiseg_task)(seed, OUTPUT_DIR) 
    for seed in random_seeds
)

valid_dfs = [res for res in results if res is not None]
if valid_dfs:
    final_df = pd.concat(valid_dfs, ignore_index=True)    
    final_output = os.path.join(OUTPUT_DIR, "all.inferred.tracts.tsv")
    final_df.to_csv(final_output, sep='\t', index=False)    
    print(f"Done! Saved to {final_output}\n")
else:
    print("No tracts found.")

print("Performance per Class without usin EM...\n\n")
res = sims.calculate_class_metrics(os.path.join(OUTPUT_DIR, "all.tracts.summary.csv"),
                                   os.path.join(OUTPUT_DIR, "all.inferred.tracts.tsv"),
                                   params['chrom_length'])

print(f"Total Analyzed Genome: {res['Total_BP']:,} bp")
print("-" * 45)
print(f"{'CLASS':<12} | {'PRECISION':<10} | {'RECALL':<10} | {'F1-SCORE':<10}")
print("-" * 45)
print(f"{'Archaic':<12} | {res['Archaic']['Precision']:<10.2%} | {res['Archaic']['Recall']:<10.2%} | {res['Archaic']['F1']:<10.4f}")
print(f"{'Modern':<12} | {res['Modern']['Precision']:<10.2%} | {res['Modern']['Recall']:<10.2%} | {res['Modern']['F1']:<10.4f}")
print("-" * 45)

Running DAIseg in parallel for 10 chromosomes...
Done! Saved to sims/all.inferred.tracts.tsv

Performance per Class without usin EM...


Total Analyzed Genome: 1,000,000,000.0 bp
---------------------------------------------
CLASS        | PRECISION  | RECALL     | F1-SCORE  
---------------------------------------------
Archaic      | 96.66%     | 85.18%     | 0.9056    
Modern       | 99.48%     | 99.90%     | 0.9969    
---------------------------------------------


In [6]:
! python ../daiseg.py run.with.EM -jsons sims/config_seed_*.json -out ./sims/all.inferred.EM.tsv

Starting Batch EM pipeline on 10 files...
Loading 10 files for Global EM & Inference...
 [obs.py] Loading BED windows...
[obs.py] Loaded 50000 windows. Processing TSV...

 [obs.py] Processing done. Aggregating results...
 Observation sequences for HMM created successfully!
Max differences in 1000bp window: 10
 [obs.py] Loading BED windows...
[obs.py] Loaded 50000 windows. Processing TSV...

 [obs.py] Processing done. Aggregating results...
 Observation sequences for HMM created successfully!
Max differences in 1000bp window: 10
 [obs.py] Loading BED windows...
[obs.py] Loaded 50000 windows. Processing TSV...

 [obs.py] Processing done. Aggregating results...
 Observation sequences for HMM created successfully!
Max differences in 1000bp window: 11
 [obs.py] Loading BED windows...
[obs.py] Loaded 50000 windows. Processing TSV...

 [obs.py] Processing done. Aggregating results...
 Observation sequences for HMM created successfully!
Max differences in 1000bp window: 10
 [obs.py] Loading BE

In [7]:
gt_path = os.path.join(OUTPUT_DIR, "all.tracts.summary.csv")
inf_path = os.path.join(OUTPUT_DIR, "all.inferred.EM.tsv")


res = sims.calculate_class_metrics(gt_path, inf_path, params['chrom_length'])

print(f"Total Analyzed Genome: {res['Total_BP']:,} bp")
print("-" * 45)
print(f"{'CLASS':<12} | {'PRECISION':<10} | {'RECALL':<10} | {'F1-SCORE':<10}")
print("-" * 45)
print(f"{'Archaic':<12} | {res['Archaic']['Precision']:<10.2%} | {res['Archaic']['Recall']:<10.2%} | {res['Archaic']['F1']:<10.4f}")
print(f"{'Modern':<12} | {res['Modern']['Precision']:<10.2%} | {res['Modern']['Recall']:<10.2%} | {res['Modern']['F1']:<10.4f}")
print("-" * 45)

Total Analyzed Genome: 1,000,000,000.0 bp
---------------------------------------------
CLASS        | PRECISION  | RECALL     | F1-SCORE  
---------------------------------------------
Archaic      | 97.02%     | 88.42%     | 0.9252    
Modern       | 99.59%     | 99.90%     | 0.9975    
---------------------------------------------
